# Part1: Data loading to TranSMART

### Load libraries

In [ ]:
import os
import tmtk
import json
import pandas as pd
from pathlib import Path

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

print('TMTK version: {}'.format(tmtk.__version__))
print('Pandas version: {}'.format(pd.__version__))

### 1.1 Data import with TMTK from template file

<br>
<font color=red>
<b>EXERCISE 1:</b> <br>
BY RUNNING THE CODE YOU'LL GET 2 ERRORS, ONE AFTER YOU RESOLVE THE OTHER. CAN YOU FIX THEM IN THE TREE STRUCTURE FILE?
</font>

In [ ]:
study = tmtk.toolbox.template_reader(template_file= 'input/template_file.xlsx')

<font color=red>
<b>EXERCISE 2:</b> <br>
LAB_DATA.xlsx CONTAINS ANOTHER RELEVANT FIELD; ADD IT TO THE TEMPLATE FILE AT THE SAME LEVEL OF NESTING AS THE OTHER VARIABLE FROM THAT FILE ("CEA (blood/serum)") </font>

### 1.2 Edit tree structure with the Arborist extension
<br>
<font color=red>
<b>EXERCISE 3:</b> <br>
THE TREE STRUCTURE FROM THE TEMPLATE DOESN'T SEEM QUITE RIGHT. EXPLORE IT USING THE ARBORIST AND MOVE THE TREE BRANCHES TO THE RIGHT PLACE - SEE PICTURES IN THE PRESENTATION FOR GUIDANCE
</font>

In [ ]:
study.call_boris()

### 1.3 Data export

In [ ]:
# Assign study id, top node (i.e. where study appears in tree structure), and privacy settings;
# by default all studies are private, you must explicitly change the security_required option to make them public
study.study_id = 'MY_STUDY' # CHANGE NAME AS YOU WISH!
study.security_required = False
print(study.study_id)
print(study.top_node)
print(study.security_required)

In [ ]:
# transmart-copy format
export = tmtk.SkinnyExport(study=study, export_directory= 'export')

In [ ]:
export.to_disk()

### 1.4 Load the data to TranSMART using transmart-copy

**NOTE: you can run the same command from the Jupyter Notebook terminal**

In [ ]:
import subprocess as sub

commands = [
    'export PGUSER=biomart_user',
    'export PGPASSWORD=biomart_user',
    'export PGHOST=transmart-database',
    'export PGPORT=5432',
    'java -jar /transmart-copy.jar -d export'
    ]

command = '; '.join(commands)
p = sub.Popen(command, stdout=sub.PIPE, stderr=sub.PIPE, shell=True)
(stdout, stderr) = p.communicate()

print('Executing command: {}'.format(command))
if p.returncode != 0:
    print('stderr:\n{}'.format(stderr.decode("utf-8")))

### 1.5 Refresh the cache before seeing the data in Glowing Bear (TranSMART visual interface)

Normally we do this automatically in ETL pipelines, or manually using dedicated tools (such as Postman)<br>

**NOTE: YOU NEED TO REPLACE THE VALUES BELOW WITH YOUR OWN LOGIN CREDENTIIALS!**

In [ ]:
# Node specific configuration
node_id = 1  # the number in the username
server_id = 1  # 1 for nodes 1-6; 2 for nodes 7-12; 3 for nodes 13-18; 4 for nodes 19-24
password = '' # fill in your password

In [ ]:
import requests
import transmart

# General configuration
user = f'user{node_id}'
client_id = f'transmart-node{node_id}'
gb_url = f'https://gb{node_id}.tuebingen{server_id}.thehyve.net'
keycloak_url = 'https://keycloak-dwh-test.thehyve.net'
keycloak_realm = 'tuebingen2019'

# Fetch offline token for API access
r = requests.post(url=f'{keycloak_url}/auth/realms/{keycloak_realm}/protocol/openid-connect/token',
                  data=dict(grant_type='password',
                            client_id=client_id,
                            scope= 'offline_access',
                            username=user,
                            password=password
                           )
                 )
if r.status_code == 200:
    offline_token = r.json().get('refresh_token')
    print('Offline token retrieved successfully')
else:
    print(f'Error: {r.status_code}')
    print(r.json())

# Create an API object to perform API queries with, using the offline token
api = transmart.get_api(
    host = f'{gb_url}/api/transmart-api-server', # URL of tranSMART server connected to your Glowing Bear
    kc_url = keycloak_url, # URL of Keycloak connected to your Glowing Bear
    client_id = client_id,
    kc_realm = keycloak_realm, # Realm in Keycloak for the tranSMART application
    offline_token = offline_token,
    print_urls = False, # Whether or not to print the API URLs used behind the scenes, to learn the API calls.
    interactive = False # Prevents pre-loading of studies etc.
)

# Common errors:
# * '401 Client Error: Unauthorized' - Wrong username/password
# * 'HTTPSConnectionPool' - Wrong tranSMART or Keycloak URL or no internet
# * '404 Client Error: Not Found' - Wrong Keycloak realm

In [ ]:
# Clear cache
api.admin.after_data_loading_update()

 -> and now continue to the Glowing Bear link on your piece of paper!

### 1.6 - Explore additional TranSMART 17.X features by loading a template with modifiers, ontology, trial visits

<br>
<font color=red>
<b>EXERCISE 4:</b> <br>
 
- Take a look at template_file_EXTRA_FEATURES.xlsx
- Explore how the sheets connect to each other:
    - How is the data in Trial visits and Ontology linked to clinical data?
    - How are modifiers used? (Hint: check for additional columns in the clincal data sheet..)
- Upload the data (cell provided below, no need to edit the tree structure this time)
- Date errors: date should be formatted as YYYY-MM-DD hh:mm:ss
- Explore the uploaded data in Glowing Bear:
    - Can you find back the information you saw in the Ontology sheet?
    - What about Trial visits and Modifiers? (Ask if you need some help!)
</font>

In [ ]:
study_extended = tmtk.toolbox.template_reader(template_file='./exercise_4B/template_file_extended.xlsx')

In [ ]:
# Assign study id, top node (i.e. where study appears in tree structure), and privacy settings;
# by default all studies are private, you must explicitly change the security_required option to make them public
study_extended.study_id = 'MY_STUDY_EXTENDED' # CHANGE NAME AS YOU WISH!
study_extended.security_required = False
print(study_extended.study_id)
print(study_extended.top_node)
print(study_extended.security_required)

In [ ]:
# transmart-copy format
export_extended = tmtk.SkinnyExport(study=study_extended, export_directory= 'export_extended')

In [ ]:
export_extended.to_disk()

In [ ]:
import subprocess as sub

commands = [
    'export PGUSER=biomart_user',
    'export PGPASSWORD=biomart_user',
    'export PGHOST=transmart-database',
    'export PGPORT=5432',
    'java -jar /transmart-copy.jar -d export_extended'
    ]

command = '; '.join(commands)
p = sub.Popen(command, stdout=sub.PIPE, stderr=sub.PIPE, shell=True)
(stdout, stderr) = p.communicate()

print('Executing command: {}'.format(command))
if p.returncode != 0:
    print('stderr:\n{}'.format(stderr.decode("utf-8")))

In [ ]:
# Node specific configuration
node_id = 1  # the number in the username
server_id = 1  # 1 for nodes 1-6; 2 for nodes 7-12; 3 for nodes 13-18; 4 for nodes 19-24
password = '' # fill in your password

In [ ]:
import requests
import transmart

# General configuration
user = f'user{node_id}'
client_id = f'transmart-node{node_id}'
gb_url = f'https://gb{node_id}.tuebingen{server_id}.thehyve.net'
keycloak_url = 'https://keycloak-dwh-test.thehyve.net'
keycloak_realm = 'tuebingen2019'

# Fetch offline token for API access
r = requests.post(url=f'{keycloak_url}/auth/realms/{keycloak_realm}/protocol/openid-connect/token',
                  data=dict(grant_type='password',
                            client_id=client_id,
                            scope= 'offline_access',
                            username=user,
                            password=password
                           )
                 )
if r.status_code == 200:
    offline_token = r.json().get('refresh_token')
    print('Offline token retrieved successfully')
else:
    print(f'Error: {r.status_code}')
    print(r.json())

# Create an API object to perform API queries with, using the offline token
api = transmart.get_api(
    host = f'{gb_url}/api/transmart-api-server', # URL of tranSMART server connected to your Glowing Bear
    kc_url = keycloak_url, # URL of Keycloak connected to your Glowing Bear
    client_id = client_id,
    kc_realm = keycloak_realm, # Realm in Keycloak for the tranSMART application
    offline_token = offline_token,
    print_urls = False, # Whether or not to print the API URLs used behind the scenes, to learn the API calls.
    interactive = False # Prevents pre-loading of studies etc.
)

# Common errors:
# * '401 Client Error: Unauthorized' - Wrong username/password
# * 'HTTPSConnectionPool' - Wrong tranSMART or Keycloak URL or no internet
# * '404 Client Error: Not Found' - Wrong Keycloak realm

In [ ]:
# Clear cache
api.admin.after_data_loading_update()